In [1]:
cd ..

/home/mehdi2277/Documents/Programming/neural_nets_research


In [2]:
import torch.optim as optim
import torch.nn as nn

from tree_to_sequence.program_datasets import *
from tree_to_sequence.tree_to_sequence import TreeToSequence
from tree_to_sequence.tree_to_sequence_attention import TreeToSequenceAttention
from tree_to_sequence.tree_encoder import TreeEncoder
from tree_to_sequence.sequence_encoder import SequenceEncoder
from tree_to_sequence.multilayer_lstm_cell import MultilayerLSTMCell
from neural_nets_library import training

In [3]:
num_vars = 10
num_ints = 11
input_as_seq = False
output_as_seq = True
one_hot = False
binarize_input = True
binarize_output = False
eos_token = True
long_base_case = True

In [ ]:
for_lambda_dset = ForLambdaDataset("ANC/VeryHard-arbitraryForList.json", binarize=binarize, 
                                   eos_token=eos_token, one_hot=one_hot, 
for_lambda_dset = ForLambdaDataset("ANC/VeryHard-arbitraryForList.json", binarize_input=binarize_input, 
                                   binarize_output=binarize_output, eos_token=eos_token, one_hot=one_hot, 
                                   long_base_case=long_base_case, input_as_seq=input_as_seq,
                                   output_as_seq=output_as_seq)

In [5]:
def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform_(param, -stdev, stdev)

In [6]:
embedding_size = 256
hidden_size = 256
nclass = num_vars + num_ints + len(lambda_ops.keys())
num_layers = 1
attention = True
alignment_size = 50
align_type = 1
encoder_input_size = num_vars + num_ints + len(for_ops.keys())

if input_as_seq:
    encoder = SequenceEncoder(encoder_input_size, hidden_size, num_layers, attention=attention, one_hot=one_hot)
else:
    encoder = TreeEncoder(encoder_input_size, hidden_size, num_layers, [1, 2], attention=attention, one_hot=one_hot)
    
if attention:
    decoder = MultilayerLSTMCell(embedding_size + hidden_size, hidden_size, num_layers)
    program_model = TreeToSequenceAttention(encoder, decoder, hidden_size, nclass, embedding_size, alignment_size=alignment_size, align_type=align_type)
else:
    decoder = MultilayerLSTMCell(embedding_size, hidden_size, num_layers)
    program_model = TreeToSequence(encoder, decoder, hidden_size, nclass, embedding_size)
    
reset_all_parameters_uniform(program_model, 0.1)
encoder.initialize_forget_bias(3)
decoder.initialize_forget_bias(3)

In [ ]:
program_model = program_model.cuda()

In [14]:
def program_accuracy(prediction, target):
    return 1 if target.tolist() == prediction else 0

def token_accuracy(prediction, target):
    pass

optimizer = optim.Adam(program_model.parameters(), lr=0.005)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=100, factor=0.8)

In [15]:
model, train_loss, validation_loss = training.train_model_anc(program_model, for_lambda_dset, optimizer,
                                                              lr_scheduler=lr_scheduler, num_epochs=3, print_every=99999999,
                                                              validation_criterion=program_accuracy, batch_size=100,
                                                              use_cuda=True, plateau_lr=True, plot_every=100)

Epoch 0/9
----------
Epoch Number: 0, Batch Number: 200, Validation Metric: 0.1750
Epoch Number: 0, Batch Number: 200, Training Loss: 8.2427
Time so far is 0m 16s
Epoch Number: 0, Batch Number: 400, Validation Metric: 0.1750
Epoch Number: 0, Batch Number: 400, Training Loss: 5.8349
Time so far is 0m 28s
Epoch Number: 0, Batch Number: 600, Validation Metric: 0.1600
Epoch Number: 0, Batch Number: 600, Training Loss: 4.2227
Time so far is 0m 41s
Epoch Number: 0, Batch Number: 800, Validation Metric: 0.2950
Epoch Number: 0, Batch Number: 800, Training Loss: 3.4238
Time so far is 0m 59s
Epoch Number: 0, Batch Number: 1000, Validation Metric: 0.3050
Epoch Number: 0, Batch Number: 1000, Training Loss: 3.3432
Time so far is 1m 14s
Epoch Number: 0, Batch Number: 1200, Validation Metric: 0.4050
Epoch Number: 0, Batch Number: 1200, Training Loss: 2.9414
Time so far is 1m 39s
Epoch Number: 0, Batch Number: 1400, Validation Metric: 0.4000
Epoch Number: 0, Batch Number: 1400, Training Loss: 2.6403
T

KeyboardInterrupt: 

In [ ]:
# train_plot_losses_old = train_plot_losses_old + train_plot_losses_new
# validation_plot_losses_old = validation_plot_losses_old + validation_plot_losses_new

import csv

torch.save(program_model, "t2s-normal-vars-model")
with open("t2s-normal-vars-train.txt", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in train_plot_losses:
        writer.writerow([val]) 
with open("t2s-normal-vars-validation.txt", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in validation_plot_losses:
        writer.writerow([val]) 